In [1]:
import sys
import io
import importlib

from simulate_game import simulate_game
from competitive_sudoku.sudoku import GameState, SudokuBoard, load_sudoku

In [2]:
#returns 0 if it was a draw, 1 if player 1 won, 2 if player 2 won
def run_simulation(first_player: str, second_player: str, time: float, board_name: str):
    module1 = importlib.import_module(first_player + '.sudokuai')
    module2 = importlib.import_module(second_player + '.sudokuai')
    player1 = module1.SudokuAI()
    player2 = module2.SudokuAI()
    player1.player_number = 1
    player2.player_number = 2
    
    # We do not check for specific ai, just give it to all of them
    player1.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    player2.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    
    board = load_sudoku(board_name)
    
    # Note for future, we do not clean up files
    
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    
    simulate_game(board, player1, player2, solve_sudoku_path='bin\\solve_sudoku.exe', calculation_time=time)
    
    sys.stdout = old_stdout
    result = buffer.getvalue()
    
    dicto = {"he game ends in a draw.": 0, "Player 1 wins the game.":1, "Player 2 wins the game.":2}
    
    if result[-24:-1] in dicto:
        winner = dicto[result[-24:-1]]
    
    else:
        print(result[-40:-1])
        winner = -1
    
    return winner

### Random player, empty-3x3.txt, varying time:

In [3]:
result3 = []

player1 = "team41_A1"
player2 = "random_player"
for time in [0.5,1,5]:
    sub_result = []
    for i in range(5):
        sub_result.append(run_simulation(player1, player2, time, "boards/empty-3x3.txt"))
        sub_result.append(run_simulation(player2, player1, time, "boards/empty-3x3.txt"))
        print(time,i, sub_result[-2], sub_result[-1])
    
    result3.append(sub_result)

print(result3)

0.5 0 1 1
0.5 1 2 2
0.5 2 2 1
0.5 3 2 2
0.5 4 2 1
1 0 2 1
1 1 2 1
1 2 1 1
1 3 2 1
1 4 2 1
5 0 1 1
5 1 1 1
5 2 1 2
5 3 2 2
5 4 1 1
[[1, 1, 2, 2, 2, 1, 2, 2, 2, 1], [2, 1, 2, 1, 1, 1, 2, 1, 2, 1], [1, 1, 1, 1, 1, 2, 2, 2, 1, 1]]
